In [1]:
%load_ext autoreload

In [2]:
%autoreload 1

In [3]:
import glob
import numpy as np
from rich.pretty import pprint
import matplotlib.pyplot as plt
import matplotlib
import hist
from hist import Hist
import mplhep as hep
import ROOT
import iminuit
import iminuit.cost
import iminuit.util
import uproot
import shutil

Welcome to JupyROOT 6.26/04


In [4]:
%aimport fit_utils
%aimport barlow_beeston

In [5]:
matplotlib.rcParams.update({'figure.max_open_warning': 0})
matplotlib.rcParams['figure.facecolor'] = 'white'
plt.style.use(hep.style.CMS)
#np.set_printoptions(suppress=True, precision=3)

In [6]:
plots = {}
plots['CR_cb'] = fit_utils.loader('CR_cb')
plots['CR_prompt'] = fit_utils.loader('CR_prompt')
plots['SR'] = fit_utils.loader('SR')

In [23]:
regions = ['CR_prompt', 'CR_cb']
processes = {
    'VV_2018': 'VV',
    'TTJets_2018': 'TT',
    'DY_NLO_all_2018': 'DY',
    'QCD_Pt_MuEnriched_2018': 'QCD',
}
signal = {
    'SUEP-m125-darkPho_2018': 'SUEP125',
}
data = 'DoubleMuon+Run2018A-UL2018_MiniAODv2-v1+MINIAOD_histograms_2018'
histogram = 'muon_pt'

with uproot.recreate('SUEP.root') as file:
    for region in regions:
        for nMuon in range(1,5):
            file[f'{region}_{nMuon}/data_obs'] = plots[region][data][histogram][:, ::sum, ::sum, nMuon*1j]
            for p in processes.keys():
                tmp_plot = plots[region][p][histogram].copy()
                value_is_negative = tmp_plot.values() < 0
                tmp_plot.variances()[value_is_negative] = 0
                tmp_plot.values()[value_is_negative] = 0
                low_unc_is_negative = (tmp_plot.values() - np.sqrt(tmp_plot.variances())) < 0
                tmp_plot.variances()[low_unc_is_negative] = (tmp_plot.values()[low_unc_is_negative])**2
                for df in range(5):
                    name = f'{region}_{nMuon}/{processes[p]}_df{df}'
                    file[name] = tmp_plot[:, ::sum, df*1j, nMuon*1j]
            for s in signal.keys():
                name = f'{region}_{nMuon}/{signal[s]}'
                file[name] = plots[region][s][histogram][:, ::sum, ::sum, nMuon*1j]                    

In [24]:
shutil.copy2('SUEP.root', '../../../combine_stuff/CMSSW_11_3_4_patch1/src/auxiliaries/')

'../../../combine_stuff/CMSSW_11_3_4_patch1/src/auxiliaries/SUEP.root'

In [24]:
pprint([p for p in plots['CR_cb'].keys() if 'SUEP' in p])

[
│   'SUEP-m125-darkPho_2018',
│   'SUEP-m125-darkPhoHad_2018',
│   'SUEP-m400-darkPho_2018',
│   'SUEP-m400-darkPhoHad_2018',
│   'SUEP-m750-darkPho_2018',
│   'SUEP-m750-darkPhoHad_2018',
│   'SUEP-m1000-darkPho_2018',
│   'SUEP-m1000-darkPhoHad_2018'
]

In [25]:
plots['CR_cb']

{'SUEP-m125-darkPho_2018': {'ht': Hist(
    Regular(30, 20, 2000, transform=log, name='ht'),
    Regular(8, 0, 8, name='nMuon_ndf'),
    Regular(8, 0, 8, name='nMuon'),
    storage=Weight()) # Sum: WeightedSum(value=1435.2, variance=3751.89) (WeightedSum(value=1526.7, variance=3991.12) with flow),
  'muon_pt': Hist(
    Regular(30, 3, 500, transform=log, name='Muon_pt'),
    IntCategory([0, 1, 3, 4, 5, 15], name='Muon_genPartFlav'),
    Regular(8, 0, 8, name='nMuon_ndf'),
    Regular(8, 0, 8, name='nMuon'),
    storage=Weight()) # Sum: WeightedSum(value=148.968, variance=389.328),
  'muon_eta': Hist(
    Regular(20, -2.4, 2.4, name='Muon_eta'),
    IntCategory([0, 1, 3, 4, 5, 15], name='Muon_genPartFlav'),
    Regular(8, 0, 8, name='nMuon_ndf'),
    Regular(8, 0, 8, name='nMuon'),
    storage=Weight()) # Sum: WeightedSum(value=148.968, variance=389.328)},
 'SUEP-m125-darkPhoHad_2018': {'ht': Hist(
    Regular(30, 20, 2000, transform=log, name='ht'),
    Regular(8, 0, 8, name='nMuon_ndf

In [20]:
print(
    plots['CR_prompt']['TTJets_2018']['ht'][:, 0, 1j].values() 
    - np.sqrt(plots['CR_prompt']['TTJets_2018']['ht'][:, 0, 1j].variances())
)

[ 0.00000000e+00  2.51416005e-03  6.65469648e-03  3.59545374e-03
 -1.23306712e-02  1.71677587e-02  1.08872562e-01  3.14947039e-01
  5.13441715e-01  1.12028571e+00  2.80772109e+00  6.17545061e+00
  1.44873717e+01  2.91528444e+01  5.03854932e+01  7.73484293e+01
  1.02568188e+02  1.14047898e+02  1.10359653e+02  9.06991891e+01
  6.68028686e+01  4.73662692e+01  3.30325222e+01  2.02116124e+01
  1.27265536e+01  7.12093780e+00  4.03206648e+00  2.14637906e+00
  7.89864985e-01  5.13965307e-01]
